# Unit 5 - Financial Planning


In [ ]:
# Initial imports

import os
import requests
import pandas as pd
from dotenv import load_dotenv
import alpaca_trade_api as tradeapi
from MCForecastTools import MCSimulation

%matplotlib inline

In [ ]:
# Load .env enviroment variables

load_dotenv()

## Part 1 - Personal Finance Planner

### Collect Crypto Prices Using the `requests` Library

In [ ]:
# Set current amount of crypto assets

my_btc = 1.2
my_eth = 5.3

In [ ]:
# Crypto API URLs

btc_url = "https://api.alternative.me/v2/ticker/Bitcoin/?convert=CAD"
eth_url = "https://api.alternative.me/v2/ticker/Ethereum/?convert=CAD"

In [ ]:
# Fetch current BTC price

btc_data = requests.get(btc_url).json()
btc_price = btc_data['data']['1']['quotes']['CAD']['price']

# Fetch current ETH price

eth_data = requests.get(eth_url).json()
eth_price = eth_data["data"]["1027"]["quotes"]["CAD"]["price"]

# Compute current value of my crpto

my_btc_value = my_btc * btc_price
my_eth_value = my_eth * eth_price

# Print current crypto wallet balance

print(f"The current value of your {my_btc} BTC is ${my_btc_value:0.2f}")
print(f"The current value of your {my_eth} ETH is ${my_eth_value:0.2f}")

###### 

### Collect Investments Data Using Alpaca: `SPY` (stocks) and `AGG` (bonds)

In [ ]:
# Current amount of shares

my_agg = 200
my_spy = 50

In [ ]:
# Set Alpaca API key and secret

alpaca_api_key = os.getenv("ALPACA_API_KEY")
alpaca_secret_key = os.getenv("ALPACA_SECRET_KEY")

# Create the Alpaca API object

alpaca = tradeapi.REST(
    alpaca_api_key,
    alpaca_secret_key,
    api_version = "v2")

In [ ]:
# Format current date as ISO format

today = pd.Timestamp("2021-03-16", tz="America/New_York").isoformat()

# Set the tickers

tickers = ["AGG", "SPY"]

# Set timeframe to '1D' for Alpaca API

timeframe = "1D"

# Get current closing prices for SPY and AGG

df_shares = alpaca.get_barset(
    tickers,
    timeframe,
    start = today,
    end = today).df

# Preview DataFrame

df_shares


In [ ]:
# Pick AGG and SPY close prices

agg_close_price = float(df_shares["AGG"]["close"])
spy_close_price = float(df_shares['SPY']["close"])

# Print AGG and SPY close prices

print(f"Current AGG closing price: ${agg_close_price}")
print(f"Current SPY closing price: ${spy_close_price}")

In [ ]:
# Compute the current value of shares

my_agg_value = my_agg * agg_close_price
my_spy_value = my_spy * spy_close_price

# Print current value of share

print(f"The current value of your {my_spy} SPY shares is ${my_spy_value:0.2f}")
print(f"The current value of your {my_agg} AGG shares is ${my_agg_value:0.2f}")

### Savings Health Analysis

In [ ]:
# Set monthly household income

monthly_income = 12000

# Create savings DataFrame

total_crypto = (my_btc_value + my_eth_value)
total_shares = (my_spy_value + my_agg_value)

savings_data = {'Amount':[total_crypto, total_shares]}

df_savings = pd.DataFrame(savings_data, index = ['Crypto','Shares'])
df_savings

In [ ]:
# Plot savings pie chart

df_savings.plot.pie(y = 'Amount', figsize = (5,5), title = "Composition of Personal Savings")


In [ ]:
# Set ideal emergency fund

emergency_fund = monthly_income * 3



# Calculate total amount of savings

total_savings = total_crypto + total_shares
total_savings

# Validate saving health

if total_savings > emergency_fund:
    print(f"Congratulations Jude! You have enough money in this fund. You have a total savings of ${total_savings:0.2f}.")
elif total_savings == emergency_fund:
    print("Congratulations on reaching your financial goal, Jude")
elif emergency_fund > total_savings:
    print(f"You are ${emergency_fund - total_savings:0.2f} away from reaching your goal")

## Part 2 - Retirement Planning

### Monte Carlo Simulation

In [ ]:
# Set start and end dates of five years back from today.
# Sample results may vary from the solution based on the time frame chosen

start_date = pd.Timestamp('2016-03-16', tz='America/New_York').isoformat()
end_date = pd.Timestamp('2021-03-16', tz='America/New_York').isoformat()

In [ ]:
# Get 5 years' worth of historical data for SPY and AGG

df_shares = alpaca.get_barset(
    tickers,
    timeframe,
    start = start_date,
    end = end_date,
    
).df

# Display sample data

df_shares.head()

In [ ]:
# Configuring a Monte Carlo simulation to forecast 30 years cumulative returns

MC_thirtyyear = MCSimulation(
    portfolio_data = df_shares,
    weights = [.40,.60],
    num_simulation = 500,
    num_trading_days = 252*30
)

In [ ]:
# Printing the simulation input data

MC_thirtyyear.portfolio_data.head()

In [ ]:
# Running a Monte Carlo simulation to forecast 30 years cumulative returns

MC_thirtyyear.calc_cumulative_return()

In [ ]:
# Plot simulation outcomes

MC_thirtyyear.plot_simulation()

In [ ]:
# Plot probability distribution and confidence intervals

MC_thirtyyear.plot_distribution()

### Retirement Analysis

In [ ]:
# Fetch summary statistics from the Monte Carlo simulation results

summary_stats = MC_thirtyyear.summarize_cumulative_return()

# Print summary statistics

print(summary_stats)

### Calculate the expected portfolio return at the 95% lower and upper confidence intervals based on a `$20,000` initial investment.

In [ ]:
# Set initial investment

initial_investment = 20000

# Use the lower and upper `95%` confidence intervals to calculate the range of the possible outcomes of our $20,000

ci_lower = round(summary_stats[8]*initial_investment,2)
ci_upper = round(summary_stats[9]*initial_investment,2)

# Print results

print(f"There is a 95% chance that an initial investment of ${initial_investment} in the portfolio"
      f" over the next 30 years will end within in the range of"
      f" ${ci_lower} and ${ci_upper}")

### Calculate the expected portfolio return at the `95%` lower and upper confidence intervals based on a `50%` increase in the initial investment.

In [ ]:
# Set initial investment

initial_investment = 20000 * 1.5

# Use the lower and upper `95%` confidence intervals to calculate the range of the possible outcomes of our $30,000

ci_lower = round(summary_stats[8]*initial_investment,2)
ci_upper = round(summary_stats[9]*initial_investment,2)

# Print results

print(f"There is a 95% chance that an initial investment of ${initial_investment} in the portfolio"
      f" over the next 30 years will end within in the range of"
      f" ${ci_lower} and ${ci_upper}")

## Optional Challenge - Early Retirement


### Five Years Retirement Option

In [ ]:
# Configuring a Monte Carlo simulation to forecast 5 years cumulative returns

MC_fiveyear = MCSimulation(
    portfolio_data = df_shares,
    weights = [.20,.80],
    num_simulation = 500,
    num_trading_days = 252*5
)

In [ ]:
# Running a Monte Carlo simulation to forecast 5 years cumulative returns

MC_fiveyear.calc_cumulative_return()

In [ ]:
# Plot simulation outcomes

MC_fiveyear.plot_simulation()


In [ ]:
# Plot probability distribution and confidence intervals

dist_plot = MC_fiveyear.plot_distribution()


In [ ]:
# Fetch summary statistics from the Monte Carlo simulation results

tbl = MC_fiveyear.summarize_cumulative_return()

# Print summary statistics

print(tbl)

In [ ]:
# Set initial investment

initial_investment = 60000

# Use the lower and upper `95%` confidence intervals to calculate the range of the possible outcomes of our $60,000

ci_lower_five = round(tbl[8]*initial_investment,2)
ci_upper_five = round(tbl[9]*initial_investment,2)

# Print results

print(f"There is a 95% chance that an initial investment of ${initial_investment} in the portfolio"
      f" over the next 5 years will end within in the range of"
      f" ${ci_lower_five} and ${ci_upper_five}")

### Ten Years Retirement Option

In [ ]:
# Configuring a Monte Carlo simulation to forecast 10 years cumulative returns

MC_tenyear = MCSimulation(
    portfolio_data = df_shares,
    weights = [.15,.85],
    num_simulation = 500,
    num_trading_days = 252*10
)

In [ ]:
# Running a Monte Carlo simulation to forecast 10 years cumulative returns

MC_tenyear.calc_cumulative_return()

In [ ]:
# Plot simulation outcomes

MC_tenyear.plot_simulation()

In [ ]:
# Plot probability distribution and confidence intervals

MC_tenyear.plot_distribution()

In [ ]:
# Fetch summary statistics from the Monte Carlo simulation results

summary_stats_tbl = MC_tenyear.summarize_cumulative_return()

# Print summary statistics

print(summary_stats_tbl)

In [ ]:
# Set initial investment

initial_investment = 60000

# Use the lower and upper `95%` confidence intervals to calculate the range of the possible outcomes of our $60,000

ci_lower_ten = round(summary_stats_tbl[8]*initial_investment,2)
ci_upper_ten = round(summary_stats_tbl[9]*initial_investment,2)


# Print results

print(f"There is a 95% chance that an initial investment of ${initial_investment} in the portfolio"
      f" over the next 10 years will end within in the range of"
      f" ${ci_lower_ten} and ${ci_upper_ten}")